# Basic data analytics with Python and SQL on the Yelp database

In the following notebook we will focus on a modified version of the Yelp database, available [here](<https://www.yelp.com/dataset>).

What we will see:

- How to connect to the Yelp database using the [psycopg](<http://initd.org/psycopg/>) library.
- Basic overview of the structure of the database.
- Queries with the aim of showing how it is possible to do basic data analytics just with SQL.

## Connection to the Yelp database

The first step is to connect to the Yelp database.

In Python, we have several libraries to connect and work with PostgreSQL. We will use psycopg. Why? Mainly because it is the most popular Python driver for PostgreSQL. It is actively maintained and supported. Hence, we import it.

In [1]:
import psycopg2

Considering that the database is running on localhost, to connect to PostgreSQL we need to know the following:

- The username you use to work with PostgreSQL. The default one is "postgres", mine is "matteobodini"
- The password given at the time of installing PostgreSQL. Mine is empty.
- The server name, or IP address on which PostgreSQL is running. We can use localhost, or it’s IP i.e., 127.0.0.1.
- The database name to which you want to connect. Here it is “yelp”.

We will: 

- use the ``connect()`` method of psycopg2, with the above arguments, to connect to PostgreSQL.

- create a cursor object, using the connection object returned by the connect method, to execute PostgreSQL queries from Python.

- close the cursor object and PostgreSQL database connection.

- catch exceptions, if any that may occur during the process.

In [2]:
try:
    connection = psycopg2.connect(user = "matteobodini",
                                  password = "",
                                  host = "localhost",
                                  port = "5432",
                                  database = "yelp")
    cursor = connection.cursor()
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to ->", record,"\n")
except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
finally:
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection closed.")

You are connected to -> ('PostgreSQL 11.6 on x86_64-apple-darwin16.7.0, compiled by Apple LLVM version 8.1.0 (clang-802.0.42), 64-bit',) 

PostgreSQL connection closed.


Some points to understand the PostgreSQL connection code in detail:

- Using ``Error`` class of psycopg, we can handle any database error and exception that may occur while working with PostgreSQL from Python. Using this approach, we can make our application robust: we can catch the database exceptions and error that may occur and maybe fix them. For instance, we can ask to the user to fill the connection data again if they are wrong.

Notice the difference between the following lines of code, when an error occurs (wrong port setting, 6432 instead of 5432).

In [3]:
try:
    connection = psycopg2.connect(user = "matteobodini",
                                  password = "",
                                  host = "localhost",
                                  port = "6432",
                                  database = "yelp")
    cursor = connection.cursor()
    cursor.execute("SELECT version();") # Print PostgreSQL version
    record = cursor.fetchone()
    print("You are connected to ->", record,"\n")
except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL: ", error)
finally:
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection closed.")

Error while connecting to PostgreSQL:  could not connect to server: Connection refused
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 6432?
could not connect to server: Connection refused
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 6432?

PostgreSQL connection closed.


In [4]:
connection = psycopg2.connect(user = "matteobodini",
                              password = "",
                              host = "localhost",
                              port = "6432",
                              database = "yelp")
cursor = connection.cursor()
cursor.execute("SELECT version();")
record = cursor.fetchone()
print("You are connected to ->", record,"\n")
if(connection):
    cursor.close()
    connection.close()
    print("PostgreSQL connection closed.")

OperationalError: could not connect to server: Connection refused
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 6432?
could not connect to server: Connection refused
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 6432?


- Using ``connection.cursor()``, we can create a cursor object which allows us to execute PostgreSQL commands through Python source code. We can create as many cursors as we want from a single connection object. Cursors created from the same connection are not isolated, i.e., any changes done to the database by a cursor are immediately visible by the other cursors.

- Using ``cursor.execute()`` we can execute a database operation or query. Execute method takes a SQL query as a parameter. We can retrieve query result using cursor methods such as ``fetchone()``, ``fetchmany()``, ``fetcthall()``. In our example, we are executing a ``SELECT version();`` query to fetch the PostgreSQL version.

## Overview of the Database

We are ready to give an overview of the database. First, we look at the relational scheme of the modified Yelp database here below:

![Drag Racing](RELschema.pdf)

We begin to explore the content of the tables. Before seeing the related code, we recall the basic structure of a query:

- Define which columns you want to see after ``SELECT``. The star ``*`` stands for "all the possible columns".

- Choose the table after ``FROM``. 

- Conditions for the data you want to use from the table(s) can be set after ``WHERE``.

The structure, and the order of the sections matter. Spaces, new lines, capital words and indentation are used to make the code easier to read.

For instance, we get part of the content of the table "business". We use the command LIMIT, that limits the number of rows in the output (in this case up to 10 rows).

In [5]:
try:
    connection = psycopg2.connect(user = "matteobodini",
                                  password = "",
                                  host = "localhost",
                                  port = "5432",
                                  database = "yelp")
    cursor = connection.cursor()
    cursor.execute("""SELECT *
                   FROM business
                   LIMIT 10;""")
    record = cursor.fetchall()
    print(record, "\n")
except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
finally:
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection closed.")

[('FYWN1wneV18bWNgQjJ2GNg', 'Dental by Design', None, '85044', '4855 E Warner Rd, Ste B9', 'D66C6A', 33.3306902, -111.9785992, 4.0, 22, True), ('He-G7vWjzVUysIKrfNbPUQ', 'Stephen Szabo Salon', None, '15317', '3101 Washington Rd', '60618F', 40.2916853, -80.1048999, 3.0, 11, True), ('KQPW8lFf1y5BT2MxiSZ3QA', 'Western Motor Vehicle', None, '85017', '6025 N 27th Ave, Ste 1', '185B21', 33.5249025, -112.1153098, 1.5, 18, True), ('8DShNS-LuFqpEWIp0HxijA', 'Sports Authority', None, '85282', '5000 Arizona Mills Cr, Ste 435', 'C15E86', 33.3831468, -111.9647254, 3.0, 9, False), ('PfOCPjBrlQAnz__NXj9h_w', 'Brick House Tavern + Tap', None, '44221', '581 Howe Ave', 'AB6205', 41.1195346, -81.4756898, 3.5, 116, True), ('o9eMRCWt5PkpLDE0gOPtcQ', 'Messina', None, '70567', 'Richterstr. 11', '483244', 48.7272, 9.14795, 4.0, 5, True), ('kCoE3jvEtg6UVz5SOD3GVw', 'BDJ Realty', 'Summerlin', '89128', '2620 Regatta Dr, Ste 102', '4598F8', 36.20743, -115.26846, 4.0, 5, True), ('OD2hnuuTJI9uotcKycxg1A', 'Soccer Z

We got the business for that the users provided a review. We notice two main points:

- Although it is not necessary, some pretty printing is needed. We can use ``pandas.read_sql()``: it takes a query and the connection to the database as arguments and returns a DataFrame containing the returned output.

- it is annoying to write every times the code needed for connecting to the database. We can wrap it in a function, like the following:

In [6]:
import pandas as pd

def query(q) :
    try:
        connection = psycopg2.connect(user = "matteobodini",
                                  password = "",
                                  host = "localhost",
                                  port = "5432",
                                  database = "yelp")
        data = pd.read_sql(q, connection)
        return data
    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)
    finally:
        if(connection):
            connection.close()

and then only call the above function:

In [7]:
query("""SELECT *
         FROM business
         LIMIT 10;""")

,id,name,neighborhood,postal,street,city,lat,lon,stars,review_count,is_active
0,FYWN1wneV18bWNgQjJ2GNg,Dental by Design,None,85044,"4855 E Warner Rd, Ste B9",D66C6A,33.330690,-111.978599,4.0,22,True
1,He-G7vWjzVUysIKrfNbPUQ,Stephen Szabo Salon,None,15317,3101 Washington Rd,60618F,40.291685,-80.104900,3.0,11,True
2,KQPW8lFf1y5BT2MxiSZ3QA,Western Motor Vehicle,None,85017,"6025 N 27th Ave, Ste 1",185B21,33.524903,-112.115310,1.5,18,True
3,8DShNS-LuFqpEWIp0HxijA,Sports Authority,None,85282,"5000 Arizona Mills Cr, Ste 435",C15E86,33.383147,-111.964725,3.0,9,False
4,PfOCPjBrlQAnz__NXj9h_w,Brick House Tavern + Tap,None,44221,581 Howe Ave,AB6205,41.119535,-81.475690,3.5,116,True
5,o9eMRCWt5PkpLDE0gOPtcQ,Messina,None,70567,Richterstr. 11,483244,48.727200,9.147950,4.0,5,True
6,kCoE3jvEtg6UVz5SOD3GVw,BDJ Realty,Summerlin,89128,"2620 Regatta Dr, Ste 102",4598F8,36.207430,-115.268460,4.0,5,True
7,OD2hnuuTJI9uotcKycxg1A,Soccer Zone,None,89128,"7240 W Lake Mead Blvd, Ste 4",4598F8,36.197484,-115.249660,1.5,9,True
8,EsMcGiZaQuG1OOvL9iUFug,Any Given Sundae,None,15090,2612 Brandt School Rd,B6F381,40.615102,-80.091349,5.0,15,True
9,TGWhGNusxyMaA4kQVBNeew,Detailing Gone Mobile,None,89014,,561403,36.055825,-115.046350,5.0,7,True


Now, we can check easily the content of the other tables:

In [8]:
query("""SELECT *
         FROM category
         LIMIT 10;""")

,name
0,Herbs & Spices
1,Bike Shop
2,Mediators
3,Cooking Schools
4,Siding
5,Hydroponics
6,Pest Control
7,Baden
8,Auto Glass Services
9,Windshield Installation & Repair


In [9]:
query("""SELECT *
         FROM city
         LIMIT 10;""")

,id,name,country
0,MI0000,Milan,IT
1,RM0000,Rome,IT
2,D66C6A,Ahwatukee,AZ
3,60618F,McMurray,PA
4,185B21,Phoenix,AZ
5,C15E86,Tempe,AZ
6,AB6205,Cuyahoga Falls,OH
7,483244,Stuttgart,BW
8,4598F8,Las Vegas,NV
9,B6F381,Wexford,PA


In [10]:
query("""SELECT *
         FROM feature
         LIMIT 10;""")

,name,description
0,AcceptsInsurance,None
1,ByAppointmentOnly,None
2,BusinessAcceptsCreditCards,None
3,BusinessParking_garage,None
4,BusinessParking_street,None
5,BusinessParking_validated,None
6,BusinessParking_lot,None
7,BusinessParking_valet,None
8,HairSpecializesIn_coloring,None
9,HairSpecializesIn_africanamerican,None


In [11]:
query("""SELECT *
         FROM friend
         LIMIT 10;""")

,reviewer_a,reviewer_b
0,JJ-aSuM4pCFPdkfoZ34q0Q,HVUAmApa0fCbHHVJ0ALshw
1,JJ-aSuM4pCFPdkfoZ34q0Q,Gl46kwumd4_f102dxgswDw
2,JJ-aSuM4pCFPdkfoZ34q0Q,i9CDu8j-qcUfZYl67WBwiw
3,JJ-aSuM4pCFPdkfoZ34q0Q,23lcgZUt6dvtMqV_AZe9KQ
4,JJ-aSuM4pCFPdkfoZ34q0Q,zacONW61-GjaCF9h5_icgA
5,JJ-aSuM4pCFPdkfoZ34q0Q,65w6GtrNNlZDowVDAYGZFQ
6,JJ-aSuM4pCFPdkfoZ34q0Q,WJeipqBVdQmVhnYrXlLb7A
7,JJ-aSuM4pCFPdkfoZ34q0Q,xtVcWcIKVORcBKhmdbrFiA
8,JJ-aSuM4pCFPdkfoZ34q0Q,7CghpxENlXnjTGOyNtRDsg
9,JJ-aSuM4pCFPdkfoZ34q0Q,SD2VfVLfSkz8m18dvo0HGA


In [12]:
query("""SELECT * FROM
         incat
         LIMIT 10;""")

,business,category
0,mmazCP1ZH0QsUqDS6OivFA,Seafood
1,Zh7k_33xMSPwm0UU7LsEIw,Education
2,EBa7-48XiJMmbbZ7VDhrjg,Shopping
3,IsUuk4HqkQaMPwpLGCHAtQ,Hair Extensions
4,WrBArrXMBdQHcdfudhneGw,Home Services
5,Fk-HB3E1MTSnekXb4ot6LA,Post Offices
6,DgzbPdeO30IRDoj3m4Sifw,Active Life
7,WzLXKlHrfN2Ml2fKPjyoRA,Food
8,uXMdcJzVt3UvV8Lum1hzFg,Gas Stations
9,9qWjF5SiK2_MGnu6wBcp-w,Food


In [13]:
query("""SELECT *
         FROM review
         LIMIT 10;""")

,stars,useful,funny,cool,reviewer,business,review_date
0,4,0,0,1,nOTl4aPC4tKHK35T3bNauQ,R3Cg6XvdBNXXNbEu6M977A,2013-07-21
1,5,1,0,1,tL2pS5UOmN6aAOi3Z-qFGg,SHj_WEN8p2U3LsglfdUY7A,2013-02-03
2,3,0,2,0,tL2pS5UOmN6aAOi3Z-qFGg,rkqTVzTkLaWSUybaWUMIdA,2014-04-29
3,3,0,0,0,tL2pS5UOmN6aAOi3Z-qFGg,aypYzV1Aad9exkitRF9UnQ,2013-04-30
4,4,1,0,0,tL2pS5UOmN6aAOi3Z-qFGg,SstJJVADGO50B8wut8M4wg,2013-05-26
5,5,0,0,0,Y76nS3L426UCz7N_1pUfUQ,J3FUxMAHfCl-WFo86u5DqA,2017-01-06
6,1,1,0,0,Tr1ulYYAHAw5FzEQQwZEJQ,pYUoaak6lIHX_8clvk5Raw,2015-05-17
7,5,2,2,1,BytRWk8X1OelSgwwfXd8Aw,9MsMQmaeluNro-k7UDIsYg,2015-07-24
8,5,7,8,8,5V8eXkTJb6IejJkMDaj_Bw,ms7cmWkPiSBDTnz-1sv2kQ,2009-09-27
9,3,2,1,1,5V8eXkTJb6IejJkMDaj_Bw,tVVZcFFt3ZlWVE7nptN3EQ,2010-06-01


In [14]:
query("""SELECT *
         FROM reviewer
         LIMIT 10;""")

,id,name,review_count,yelp_since,useful,funny,cool,fans,average_stars
0,JJ-aSuM4pCFPdkfoZ34q0Q,Chris,10,2013-09-24,0,None,0,0,3.70
1,uUzsFQn_6cXDh6rPNGbIFA,Tiffy,1,2017-03-02,0,None,0,0,2.00
2,mBneaEEH5EMyxaVyqS-72A,Mark,6,2015-03-13,0,None,0,0,4.67
3,W5mJGs-dcDWRGEhAzUYtoA,Evelyn,3,2016-09-08,0,None,0,0,4.67
4,4E8--zUZO1Rr1IBK4_83fg,Lisa,11,2012-07-16,4,None,0,0,3.45
5,Ob-2oGBQ7rwwYwUvhmnf7g,B,9,2012-05-01,0,None,0,0,4.78
6,JaTVvKsBl0bHHJEpESn4pQ,Peter,2,2013-03-23,0,None,0,0,5.00
7,Ykj0DVsz0c6rX9ghjd0hDg,Colleen,1,2010-10-10,0,None,0,0,1.00
8,kmyEPfKnHQJdTceCdoyMQg,A,7,2012-10-16,0,None,0,0,4.29
9,H54pA7YHfjl8IjhHAfdXJA,Chad,3,2010-06-25,0,None,0,0,5.00


In [15]:
query("""SELECT * 
         FROM schedule
         LIMIT 10;""")

,business,day,open,close
0,FYWN1wneV18bWNgQjJ2GNg,monday,07:30:00,17:00:00
1,FYWN1wneV18bWNgQjJ2GNg,tuesday,07:30:00,17:00:00
2,FYWN1wneV18bWNgQjJ2GNg,wednesday,07:30:00,17:00:00
3,FYWN1wneV18bWNgQjJ2GNg,thursday,07:30:00,17:00:00
4,FYWN1wneV18bWNgQjJ2GNg,friday,07:30:00,17:00:00
5,He-G7vWjzVUysIKrfNbPUQ,monday,09:00:00,20:00:00
6,He-G7vWjzVUysIKrfNbPUQ,tuesday,09:00:00,20:00:00
7,He-G7vWjzVUysIKrfNbPUQ,wednesday,09:00:00,20:00:00
8,He-G7vWjzVUysIKrfNbPUQ,thursday,09:00:00,20:00:00
9,He-G7vWjzVUysIKrfNbPUQ,friday,09:00:00,16:00:00


In [16]:
query("""SELECT *
         FROM services
         LIMIT 10;""")

,business,feature
0,936_FRHqiTCp6c-p6U0a2Q,ByAppointmentOnly
1,FU0qfg5GsrLpzPvOGqV2zg,ByAppointmentOnly
2,lCxnFlMO3Q8NHDPNIYFxhg,ByAppointmentOnly
3,Yivn60GjQTIIsuRcbLeW7A,ByAppointmentOnly
4,tvgfhAYM5UorNE_9F9WywA,ByAppointmentOnly
5,c46Geowa3RCGRWpQKtMkdA,ByAppointmentOnly
6,44fINSiJC7WyfiptRC07Lw,ByAppointmentOnly
7,zAFThvKpX7RMQec_9Y0U9A,ByAppointmentOnly
8,LjesbXmGlPw1--DHDgCHDw,ByAppointmentOnly
9,p3mhi3IWiXFIrc_CYmJmeA,ByAppointmentOnly
